In [5]:
import pandas as pd
import numpy as np
import re
from pathlib import Path

# ---------------------------
# Utilities
# ---------------------------
def clean_text(text: str):
    if not isinstance(text, str):
        return text
    replacements = {
        "â€™": "'",
        "â€œ": '"',
        "â€": '"',
        "â€“": "–",
        "â€”": "–",
        "â€¦": "...",
        "Â": ""
    }
    for bad, good in replacements.items():
        text = text.replace(bad, good)
    text = re.sub(r"\s+", " ", text).strip()
    return text

def normalize_name(s):
    if not isinstance(s, str):
        return ""
    s = s.strip().lower()
    s = re.sub(r"\s+", " ", s)
    s = re.sub(r"[^\w\s\-&/]+$", "", s)
    return s

def safe_sheet_name(name, max_len=31):
    return name[:max_len]

def infer_sentiment_column(df):
    """
    Try to find a usable sentiment column.
    Preference order:
      - columns containing 'sentiment' or 'compound'
    Returns (colname, scale) where scale is one of {"pm1","01","15"} meaning:
      pm1: values roughly in [-1,1]
      01 : values in [0,1]
      15 : values in [1,5] (ratings)
    """
    candidates = []
    for c in df.columns:
        lc = c.lower()
        if "sent" in lc or "compound" in lc or lc in {"sentiment", "compound"}:
            if pd.api.types.is_numeric_dtype(df[c]):
                nonnull = df[c].notna().mean()
                if nonnull > 0.5:  # at least half filled
                    candidates.append(c)
    if not candidates:
        raise ValueError("No numeric sentiment-like column found in the sentence-level table.")

    # pick the one with largest non-null coverage
    best = max(candidates, key=lambda x: df[x].notna().mean())
    s = df[best].dropna()
    vmin, vmax = s.min(), s.max()

    if vmin < 0 and vmax > 0:
        scale = "pm1"           # typical VADER/transformer polarity
    elif 0 <= vmin and vmax <= 1.00001:
        scale = "01"            # probabilities
    elif 1 <= vmin and vmax <= 5.00001:
        scale = "15"            # star ratings
    else:
        # fallback: assume centered at 0 if it crosses 0; else treat as 0..1-like
        scale = "pm1" if (vmin < 0 and vmax > 0) else "01"
    print(f"Using sentiment column: '{best}' (min={vmin:.3f}, max={vmax:.3f}, scale={scale})")
    return best, scale

def label_sentiment_series(s, scale):
    """
    Convert a numeric sentiment series to 'Pro' / 'Con' / 'Neutral' based on scale.
    Your requirement: Pros = positive only, Cons = negative only.
    """
    s = pd.to_numeric(s, errors="coerce")
    if scale == "pm1":   # [-1, 1]
        pro = s > 0
        con = s < 0
    elif scale == "01":  # [0, 1]
        pro = s >= 0.55
        con = s <= 0.45
    elif scale == "15":  # [1, 5]
        pro = s >= 4.0
        con = s <= 2.0
    else:
        pro = s > 0
        con = s < 0
    out = pd.Series("Neutral", index=s.index)
    out[pro] = "Pro"
    out[con] = "Con"
    return out

# ---------------------------
# Load data
# ---------------------------
product_df = pd.read_excel("Topic_sentiment_product_level_table_FINAL.xlsx",
                           sheet_name="Topic sentiment product level")
sentence_df = pd.read_excel("Topic_sentiment_sentence_level_table.xlsx",
                            sheet_name="Topic sentiment sentence level")

print("Product-level columns:", product_df.columns.tolist())
print("Sentence-level columns:", sentence_df.columns.tolist())

# Normalize product names
product_df["product_name_cleaned_norm"] = product_df["product_name_cleaned"].apply(normalize_name)
sentence_df["product_name_cleaned_norm"] = sentence_df["product_name_cleaned"].apply(normalize_name)

# Build string IDs for joins
product_df["Theme_id_str"] = product_df["Theme_id"].astype(str)
sentence_df["final_subtopic_id_str"] = sentence_df["final_subtopic_id"].astype(str)

# Normalize topic names for name-based fallback
product_df["Theme_name_norm"] = product_df["Theme_name"].apply(normalize_name)
sentence_df["final_subtopic_name_norm"] = sentence_df["final_subtopic_name"].apply(normalize_name)

# ---------------------------
# Topic ranking (composite)
# ---------------------------
prom_col = "FreqWeight (Topic Prominence)"
sent_col_prod = "Subtopic_AvgSentiment"

prod_rank = product_df.copy()
prod_rank["Topic_Score"] = 0.5 * prod_rank[sent_col_prod] + 0.5 * prod_rank[prom_col]
prod_rank = prod_rank.sort_values(["product_name_cleaned", "Topic_Score"], ascending=[True, False])

# Keep necessary columns
prod_rank_min = prod_rank[[
    "product_name_cleaned", "subtopic_id", "subtopic_name",
    prom_col, sent_col_prod, "Topic_Score"
]].rename(columns={
    "subtopic_id": "subtopic_id_str",
    "subtopic_name": "subtopic_name_norm"
})

# Convert types for matching
prod_rank_min["subtopic_id_str"] = prod_rank_min["subtopic_id_str"].astype(str)
sentence_df["final_subtopic_id_str"] = sentence_df["final_subtopic_id"].astype(str)

# Normalize topic names
prod_rank_min["subtopic_name_norm"] = prod_rank_min["subtopic_name_norm"].apply(normalize_name)
sentence_df["final_subtopic_name_norm"] = sentence_df["final_subtopic_name"].apply(normalize_name)

# ---------------------------
# Merge using subtopic_id first
# ---------------------------
merged = sentence_df.merge(
    prod_rank_min[["product_name_cleaned", "subtopic_id_str", "Topic_Score"]],
    left_on=["product_name_cleaned", "final_subtopic_id_str"],
    right_on=["product_name_cleaned", "subtopic_id_str"],
    how="left"
)
id_cov = merged["Topic_Score"].notna().mean()
print(f"ID-merge coverage: {id_cov*100:.1f}%")

# ---------------------------
# Fallback: merge by subtopic_name
# ---------------------------
if id_cov < 0.9:
    fb = sentence_df.merge(
        prod_rank_min[["product_name_cleaned", "subtopic_name_norm", "Topic_Score"]],
        left_on=["product_name_cleaned", "final_subtopic_name_norm"],
        right_on=["product_name_cleaned", "subtopic_name_norm"],
        how="left"
    )
    merged.loc[merged["Topic_Score"].isna(), "Topic_Score"] = fb["Topic_Score"].values
    print(f"After name-fallback coverage: {merged['Topic_Score'].notna().mean()*100:.1f}%")

# ---------------------------
# Sentiment labeling (Pros/Cons)
# ---------------------------
sent_col_sent = "sentiment_score"
merged["Category"] = merged[sent_col_sent].apply(lambda x: "Pro" if x > 0 else ("Con" if x < 0 else "Neutral"))

print("\nLabel distribution:")
print(merged["Category"].value_counts())

# ---------------------------
# Extract top 5 subtopics per product for Pros and Cons separately
# ---------------------------
top_n_topics = 5

# For Pros: higher average sentiment first (tie-break with prominence)
pro_rank = (
    prod_rank_min
    .sort_values(["product_name_cleaned", sent_col_prod, prom_col],
                 ascending=[True, False, False])
)
top_topics_pro = (
    pro_rank.groupby("product_name_cleaned")["subtopic_id_str"]
    .apply(lambda s: list(s.head(top_n_topics)))
    .to_dict()
)

# For Cons: lower average sentiment first (more negative), tie-break with prominence
con_rank = (
    prod_rank_min
    .sort_values(["product_name_cleaned", sent_col_prod, prom_col],
                 ascending=[True, True, False])
)
top_topics_con = (
    con_rank.groupby("product_name_cleaned")["subtopic_id_str"]
    .apply(lambda s: list(s.head(top_n_topics)))
    .to_dict()
)

def _unique_preserve(seq):
    seen = set()
    out = []
    for x in seq:
        if x not in seen:
            seen.add(x)
            out.append(x)
    return out

# ---------------------------
# Extract representative sentences (using Pro/Con topic sets separately)
# ---------------------------
top_n_sentences = 5
results = []

for prod in product_df["product_name_cleaned"].unique():
    topics_pro = top_topics_pro.get(prod, [])
    topics_con = top_topics_con.get(prod, [])
    topics_combined = _unique_preserve(topics_pro + topics_con)  # keep original column name downstream

    sub = merged[merged["product_name_cleaned"] == prod].copy()

    # Pros: from pro topic set, sort by Topic_Score desc then sentiment desc
    pros = sub[
        (sub["Category"] == "Pro") &
        (sub["final_subtopic_id_str"].isin(topics_pro))
    ].sort_values(by=["Topic_Score", sent_col_sent], ascending=[False, False])

    # Cons: from con topic set, sort by Topic_Score desc then sentiment asc (strong negatives)
    cons = sub[
        (sub["Category"] == "Con") &
        (sub["final_subtopic_id_str"].isin(topics_con))
    ].sort_values(by=["Topic_Score", sent_col_sent], ascending=[False, True])

    pros_list = pros["sentence_text"].dropna().apply(clean_text).unique().tolist()[:top_n_sentences]
    cons_list = cons["sentence_text"].dropna().apply(clean_text).unique().tolist()[:top_n_sentences]

    results.append({
        "product_name_cleaned": prod,
        "Top_ProsCons_Topics": ", ".join(topics_combined),  # keep existing column
        "Pros": pros_list,
        "Cons": cons_list,
        "Pros Summary": " ".join(pros_list),
        "Cons Summary": " ".join(cons_list),
    })

summary_df = pd.DataFrame(results)

print("\nSample output:")
display(summary_df.head())


Product-level columns: ['product_name_cleaned', 'Theme_id', 'Theme_name', 'subtopic_id', 'subtopic_name', 'FreqWeight (Topic Prominence)', 'Subtopic_AvgSentiment', 'SubTopicScore', 'ProductAvgSentiment', 'RelativeSubtopicSentiment']
Sentence-level columns: ['Unnamed: 0.1', 'Unnamed: 0', 'author_id', 'rating', 'is_recommended', 'helpfulness', 'total_feedback_count', 'total_neg_feedback_count', 'total_pos_feedback_count', 'submission_time', 'review_text', 'review_title', 'skin_tone', 'eye_color', 'skin_type', 'hair_color', 'product_id', 'product_name', 'brand_name', 'price_usd', 'primary_category', 'secondary_category', 'tertiary_category', 'variation_type', 'variation_value', 'variation_desc', 'review_seq_id', 'product_name_cleaned', 'review_count', 'rank_in_brand', '_informative_', 'author_id_f', 'sentence_text', 'sentence_index', 'n_sentences_in_review', 'sentence_char_len', 'clean_sentence_lda', 'global_topic_id', 'global_topic_name', 'final_subtopic_id', 'final_subtopic_name', 'max_

,product_name_cleaned,Top_ProsCons_Topics,Pros,Cons,Pros Summary,Cons Summary
0,A-Passioni Retinol Cream,"300, 100, 400, 500, 504, 402, 403, 200, 601, 602",[Definitely the most effective skincare produc...,[It was very disappointing given the high pric...,Definitely the most effective skincare product...,It was very disappointing given the high price...
1,AHA 30% + BHA 2% Exfoliating Peeling Solution,"300, 402, 401, 100, 200, 601, 602, 600, 501, 500",[I can't wait to see how much more my skin imp...,[extraordinarily terrible for my skin i am ble...,I can't wait to see how much more my skin impr...,extraordinarily terrible for my skin i am blea...
2,Acne Solutions Cleansing Foam,"300, 200, 504, 101, 600, 402, 401, 403, 601, 400",[It is the best for get my face clean and keep...,"[Cleanses fine but very unpleasant smell., The...",It is the best for get my face clean and keepi...,Cleanses fine but very unpleasant smell. There...
3,All About Clean Liquid Facial Soap,"200, 300, 504, 600, 503, 401, 403, 400, 402, 100",[This stuff feels so great when you wash your ...,[This dries my skin out so badly that I have m...,This stuff feels so great when you wash your f...,This dries my skin out so badly that I have mo...
4,Alpha Beta Extra Strength Daily Peel Pads,"100, 300, 101, 504, 503, 403, 400, 402, 401, 200",[Bright and fresh face without leaving my skin...,"[A lot of money to pay for nothing., I hate lo...",Bright and fresh face without leaving my skin ...,A lot of money to pay for nothing. I hate losi...


In [7]:
# ---------------------------
# Export to Excel
# ---------------------------
out_dir = Path("task3_outputs")
out_dir.mkdir(exist_ok=True)
excel_path = out_dir / "Task3_Review_Summarization_Extractive_v2.xlsx"

with pd.ExcelWriter(excel_path, engine="openpyxl") as writer:
    prod_rank_min.to_excel(writer, sheet_name=safe_sheet_name("Topic Ranking"), index=False)
    summary_df.to_excel(writer, sheet_name=safe_sheet_name("Pros_Cons_Summary"), index=False)

print(f"✅ Saved: {excel_path}")


✅ Saved: task3_outputs\Task3_Review_Summarization_Extractive_v2.xlsx


In [11]:
# ============================================
# 1) Imports & Config
# ============================================
import re, ast
import pandas as pd
from pathlib import Path

INPUT_XLSX  = "Task3_Review_Summarization_Extractive_v2.xlsx"
INPUT_SHEET = "Pros_Cons_Summary"
OUTPUT_XLSX = "Task3_Review_Summarization_Generative_RuleBased_v2.xlsx"

# ============================================
# 2) Load Data (robust to list-or-string)
# ============================================
df = pd.read_excel(INPUT_XLSX, sheet_name=INPUT_SHEET)

for col in ["product_name_cleaned","Pros","Cons","Pros Summary","Cons Summary"]:
    if col not in df.columns: df[col] = ""

def safe_list(x):
    if isinstance(x, list): return x
    if isinstance(x, str) and x.strip().startswith("["):
        try:
            v = ast.literal_eval(x)
            return v if isinstance(v, list) else []
        except Exception:
            return []
    return []

def text_to_sentences(txt):
    txt = str(txt or "").strip()
    if not txt: return []
    parts = re.split(r"(?<=[.!?])\s+", txt)
    return [p.strip() for p in parts if p.strip()]

def get_pros_list(row):
    L = safe_list(row.get("Pros", []))
    return (L[:5] if L else text_to_sentences(row.get("Pros Summary", ""))[:5])

def get_cons_list(row):
    L = safe_list(row.get("Cons", []))
    return (L[:5] if L else text_to_sentences(row.get("Cons Summary", ""))[:5])

# ============================================
# 3) Cleaning & Normalization
# ============================================
MOJIBAKE = {"â€™":"'", "â€œ":'"', "â€":'"', "â€“":"-", "â€”":"-", "â€¦":"...", "Â":""}

DROP_PATTERNS = [
    r"\bhopefully\b",
    r"\bfind your\b",
    r"\bworks for you\b",
    r"\bhighly recommend(ed)?\b",
    r"\bso worth it\b",
    r"^wow\b",
]

def normalize_sentence(s: str) -> str:
    if not isinstance(s, str): return ""
    for k,v in MOJIBAKE.items(): s = s.replace(k,v)
    s = re.sub(r"^\s*\[(?:pros?|cons?)\]\s*:?[\s\-]*","", s, flags=re.I)   # strip [Pros]/[Cons]
    s = re.sub(r"\s+", " ", s).strip()
    s = re.sub(r"[!?]{2,}", "!", s)
    s = re.sub(r"\.\s*\.+", ".", s)
    if s and s[0].isalpha(): s = s[0].upper() + s[1:]
    if re.search(r"[A-Za-z0-9)]$", s): s += "."
    return s

def is_informative(s: str) -> bool:
    t = re.sub(r"[^a-z0-9]+","", s.lower())
    if len(t) < 15: return False
    if any(re.search(p, s.lower()) for p in DROP_PATTERNS): return False
    return True

def to_aggregate_voice(s: str) -> str:
    s = re.sub(r"\bI\b", "users", s)
    s = re.sub(r"\bI'm\b", "users are", s, flags=re.I)
    s = re.sub(r"\bmy\b", "their", s, flags=re.I)
    s = re.sub(r"\bme\b", "them", s, flags=re.I)
    s = re.sub(r"\bwe\b", "users", s, flags=re.I)
    s = re.sub(r"\bours\b", "their", s, flags=re.I)
    # soften superlatives
    s = re.sub(r"\bmy all[- ]time favorite\b", "a customer favorite", s, flags=re.I)
    s = re.sub(r"\bbest retinol product\b", "one of the best retinol products", s, flags=re.I)
    return s

def prepare_sentences(raw_list):
    out = []
    for s in raw_list:
        s = normalize_sentence(s)
        if not s or not is_informative(s): continue
        s = to_aggregate_voice(s)
        out.append(s)
    # de-duplicate (case/punct insensitive)
    seen, uniq = set(), []
    for s in out:
        key = re.sub(r"[^a-z0-9]+","", s.lower())
        if key not in seen:
            seen.add(key)
            uniq.append(s)
    return uniq[:5]

# ============================================
# 4) Lightweight Theme Detection
# ============================================
# We flag coarse themes to craft natural prose.
PROS_THEMES = {
    "effectiveness": [r"improv", r"results?", r"worked", r"difference", r"clear", r"texture", r"softer", r"glow", r"acne", r"breakout"],
    "quick_results": [r"after (a few|two|couple) (uses|weeks|days)", r"quick", r"fast", r"within"],
    "favorite_best": [r"favorite", r"one of the best", r"best", r"holy grail"],
    "feel_texture": [r"light(weight)?", r"absorb", r"non[- ]greasy", r"smooth", r"gentle"],
}

CONS_THEMES = {
    "dry_peel": [r"dry", r"peel", r"flak", r"dehydrat", r"tight"],
    "red_irritate": [r"red", r"irritat", r"sensitive", r"burn"],
    "breakouts": [r"pimple", r"breakout", r"acne", r"bumpy"],
    "strong_react": [r"worst", r"severe", r"react", r"rash"],
}

def flag_themes(sentences, theme_dict):
    flags = {k: False for k in theme_dict}
    lower = " ".join(sentences).lower()
    for k, pats in theme_dict.items():
        flags[k] = any(re.search(p, lower) for p in pats)
    return flags

# ============================================
# 5) Natural Renderers (phrase banks)
# ============================================
def render_pros(sentences):
    if not sentences: return ""
    flags = flag_themes(sentences, PROS_THEMES)
    parts = []

    if flags["favorite_best"]:
        parts.append("many consider it a customer favorite and among the better over-the-counter options")
    if flags["effectiveness"]:
        parts.append("it delivers noticeable improvements to skin texture and clarity")
    if flags["quick_results"]:
        parts.append("with some seeing results within a few uses")
    if flags["feel_texture"]:
        parts.append("and the texture feels lightweight and absorbs well")

    if not parts:
        # fallback to first two sentences paraphrased lightly
        s1 = re.sub(r"\.\s*$","", sentences[0])
        s2 = re.sub(r"\.\s*$","", sentences[1]) if len(sentences)>1 else ""
        out = f"Customers appreciated that {s1[0].lower()+s1[1:]}."
        if s2:
            out += f" In addition, {s2[0].lower()+s2[1:]}."
        return out

    # build up to two sentences
    sent1 = "Customers appreciated that " + parts[0] + "."
    extra = parts[1:]
    if extra:
        # join 1–2 extra parts naturally
        if len(extra) == 1:
            sent2 = extra[0].capitalize() + "."
        else:
            sent2 = (extra[0].capitalize() + ", " + extra[1] + ".")
        return f"{sent1} {sent2}"
    return sent1

NEG_OPENERS = [
    "Some users experienced",
    "Several reviewers reported",
    "A number of customers mentioned",
]

def render_cons(sentences):
    if not sentences: return ""
    flags = flag_themes(sentences, CONS_THEMES)
    items = []

    if flags["dry_peel"]:
        items.append("dryness or peeling")
    if flags["red_irritate"]:
        items.append("redness or irritation")
    if flags["breakouts"]:
        items.append("next-day breakouts or bumpy skin")
    if flags["strong_react"]:
        items.append("strong adverse reactions in a few cases")

    opener = NEG_OPENERS[min(len(sentences)-1, len(NEG_OPENERS)-1)]
    if not items:
        # fallback to first two negatives
        s1 = re.sub(r"\.\s*$","", sentences[0])
        s2 = re.sub(r"\.\s*$","", sentences[1]) if len(sentences)>1 else ""
        if s2:
            return f"{opener} {s1[0].lower()+s1[1:]}, and {s2[0].lower()+s2[1:]}."
        else:
            return f"{opener} {s1[0].lower()+s1[1:]}."
    else:
        if len(items) == 1:
            return f"{opener} {items[0]}."
        if len(items) == 2:
            return f"{opener} {items[0]} and {items[1]}."
        # 3+ items
        return f"{opener} {', '.join(items[:-1])}, and {items[-1]}."

def strip_lead_for_overall(text, lead="Customers appreciated that "):
    t = text.strip()
    if t.lower().startswith(lead.lower()):
        t = t[len(lead):].strip()
        if t and t[0].isalpha(): t = t[0].lower()+t[1:]
    return t

def tone_from_lengths(pros_txt, cons_txt):
    lp, lc = len(pros_txt.split()), len(cons_txt.split())
    total = max(lp+lc, 1)
    r = lp/total
    if r >= 0.65: return "positive"
    if r <= 0.35: return "negative"
    return "balanced"

def render_overall(pros_par, cons_par, tone):
    pros_body = strip_lead_for_overall(pros_par) if pros_par else ""
    cons_body = cons_par.strip()
    if pros_body and cons_body:
        if tone == "positive":
            return f"Overall, users found that {pros_body} However, {cons_body[0].lower()+cons_body[1:]}"
        elif tone == "negative":
            return f"Overall, users found that {cons_body} On the plus side, {pros_body[0].lower()+pros_body[1:]}"
        else:
            return f"Overall, users found that {pros_body} However, {cons_body[0].lower()+cons_body[1:]}"
    return pros_par or cons_par

# ============================================
# 6) Generate & Export
# ============================================
pros_out, cons_out, overall_out, tones = [], [], [], []

for _, row in df.iterrows():
    pros_raw = get_pros_list(row)
    cons_raw = get_cons_list(row)

    pros_clean = prepare_sentences(pros_raw)
    cons_clean = prepare_sentences(cons_raw)

    pros_par = render_pros(pros_clean)
    cons_par = render_cons(cons_clean)
    tone = tone_from_lengths(" ".join(pros_clean), " ".join(cons_clean))
    overall_par = render_overall(pros_par, cons_par, tone)

    pros_out.append(pros_par)
    cons_out.append(cons_par)
    overall_out.append(overall_par)
    tones.append(tone)

df["Pros_Generated"] = pros_out
df["Cons_Generated"] = cons_out
df["Overall_Generated"] = overall_out
df["Overall_Tone"] = tones

cols_keep = [c for c in [
    "product_name_cleaned",
    "Pros","Cons","Pros Summary","Cons Summary",
    "Pros_Generated","Cons_Generated","Overall_Generated","Overall_Tone"
] if c in df.columns]

out_path = Path(OUTPUT_XLSX)
with pd.ExcelWriter(out_path, engine="openpyxl") as w:
    df[cols_keep].to_excel(w, index=False, sheet_name="Generated_Summaries")

print(f"✅ Saved: {out_path.resolve()}")


✅ Saved: C:\Users\zongy\OneDrive\Desktop\SMU\ISSS609 - Text Analytics and Applications\Proj\Task3_Review_Summarization_Generative_RuleBased_v2.xlsx
